# TmVO4 neutrons data analysis
Fit neutrons diffraction peaks measured on TmVO4 at SNS on 2019-02-14 in order to extract the orthorhombic distortion as a function of magnetic field

## Import modules

In [24]:
from mpl_toolkits.mplot3d import Axes3D# for 3D plotting
import matplotlib.tri as mtri# for triangulation of unevenly separated data, like our magnetic field data

import copy as cp, numpy as np, pandas as pd, pickle, os, re
import importlib, sys 
import matplotlib
from matplotlib import cm, pyplot as plt, rcsetup, rc, rcParams# import matplotlib.pyplot as plt
# cm stands for colormap
from matplotlib.ticker import LogLocator, LinearLocator, FormatStrFormatter
from scipy.interpolate import griddata
from scipy.special import erfc, exp1

from lmfit import minimize, Model, Parameters, report_fit
# import pytest

### Change default plotting parameters
see https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.figure.html

In [19]:
rcParams["figure.figsize"] = np.multiply([6.4, 4.8],0.5)# default is [6.4, 4.8]

## Import data

### Coarse data

In [4]:
tempPath = r'C:\Users\Pierre\Desktop\Postdoc\TmVO4\TmVO4_neutrons\2019-02_ORNL_Corelli\2019-02-14'
os.chdir(tempPath)
(_, _, filenames) = next(os.walk(tempPath))# the walk() function lists the content of the directory that it is given as argument,
# and of its subdirectories; the next() function returns the next output of the walk() function;
# when used only once, it returns only the first output which is the content of the parent directory, 
# listed as a tuple of the form (dirpath, dirnames, filenames)
filenames = [filename for filename in filenames if 'p6K_' in filename]# keep only files that match the string pattern of datafilestempList = [None]*len(filenames)# Pre-allocate temporary list to store the data that will then be converted into a Pandas DataFrame

tempList = [None]*len(filenames)
for idx, filename in enumerate(filenames):# For each data file
    H = float(re.split('p6K_(\w*)T\w*.txt',filenames[idx])[1].replace('p','.'))# Extract value of magnetic field from filename
    tempDF = pd.read_csv(filenames[idx],names=["hh0","I","dI"],skiprows=2,delimiter=',')# import data as a Pandas DataFrame
    tempList[idx] = {'filename': filenames[idx],# Store into a dictionary, which is itself an element of tempList: the filename,
                   'H (T)': H,# value of magnetic field,
                   'T (K)': 0.6,# temperature,
                   'spectra': tempDF# and data stored as Pandas dataframe
                   }
coarseData = pd.DataFrame(tempList)# Convert the list of dictionaries into a Pandas DataFrame
del idx, H, tempPath, filename, filenames, tempList, tempDF# delete temporary variables after use 
coarseData# Show the resulting DataFrame

,filename,H (T),T (K),spectra
0,p6K_1T.txt,1.00,0.6,hh0 I dI 0 -11.99...
1,p6K_1T_new.txt,1.00,0.6,hh0 I dI 0 -11.99...
2,p6K_p05T.txt,0.05,0.6,hh0 I dI 0 -11.9975...
3,p6K_p4T.txt,0.40,0.6,hh0 I dI 0 -11.997...
4,p6K_p5T.txt,0.50,0.6,hh0 I dI 0 -11.99...
5,p6K_p6T.txt,0.60,0.6,hh0 I dI 0 -11.99...
6,p6K_p77T.txt,0.77,0.6,hh0 I dI 0 -11.99...
7,p6K_p7T.txt,0.70,0.6,hh0 I dI 0 -11.99...
8,p6K_p83T.txt,0.83,0.6,hh0 I dI 0 -11.99...
9,p6K_p97T.txt,0.97,0.6,hh0 I dI 0 -11.99...


In [5]:
coarseData.spectra[3].head()# Check the content of individual datasets after importation

,hh0,I,dI
0,-11.9975,142028.0,52309.7
1,-11.9925,220372.0,62925.4
2,-11.9875,220002.0,62818.9
3,-11.9825,518428.0,92939.8
4,-11.9775,817317.0,127501.0


### Fine data

In [6]:
tempPath = r'C:\Users\Pierre\Desktop\Postdoc\TmVO4\TmVO4_neutrons\2019-02_ORNL_Corelli\2019-02-14\p6K\linecut_f'
os.chdir(tempPath)
fieldInfo = pd.read_csv('field_info.txt',header=None,names=['File #','T (K)','H (T)','Proton charge'],delimiter=' ')
fieldInfo['File #'] = fieldInfo['File #'].astype('int')# replace file number type from float to int
fieldInfo.head()
# fieldInfo['File #'].dtype# check that the change of datatype is effective

,File #,T (K),H (T),Proton charge
0,88631,0.605973,0.000000,1.312139
1,88632,0.613334,0.049998,0.800119
2,88633,0.605727,0.099996,0.083534
3,88634,0.625493,0.099996,0.800739
4,88635,0.618220,0.150005,0.801294


In [7]:
tempList = [None]*len(fieldInfo)# Preallocate list to store the data that will then be converted into a Pandas DataFrame
for idx in range(len(fieldInfo)):# For each data file
    filename = ''.join(['HH0_',str(fieldInfo['File #'][idx]),'.txt'])
#     print(filename)
    tempDF = pd.read_csv(filename,names=["hh0","I","dI"],skiprows=2,delimiter=',')# Import data as a Pandas DataFrame
    tempList[idx] = {'filename': filename,# Store into a dictionary, which is itself an element of dfList: the filename,
                     'T (K)': fieldInfo['T (K)'][idx],# temperature,
                     'H (T)': fieldInfo['H (T)'][idx],# value of magnetic field,
                     'Proton charge': fieldInfo['Proton charge'][idx],# proton charge,
                     'spectra': tempDF# and data,
                     }
linecut_f_raw = pd.DataFrame(tempList)# Convert the list of dictionaries into a Pandas DataFrame
linecut_f_raw.head()# Show the resulting DataFrame

,filename,T (K),H (T),Proton charge,spectra
0,HH0_88631.txt,0.605973,0.000000,1.312139,hh0 I dI 0 -12.99880 0.0 ...
1,HH0_88632.txt,0.613334,0.049998,0.800119,hh0 I dI 0 -12.99880 0.0 ...
2,HH0_88633.txt,0.605727,0.099996,0.083534,hh0 I dI 0 -12.99880 0.0 ...
3,HH0_88634.txt,0.625493,0.099996,0.800739,hh0 I dI 0 -12.99880 0.0 ...
4,HH0_88635.txt,0.618220,0.150005,0.801294,hh0 I dI 0 -12.99880 0.0 ...


In [8]:
del idx, tempPath, filename, tempList, tempDF# delete temporary variables after use 

In [9]:
linecut_f_raw['spectra'][3].loc[180:185]#.head()# Check the content of individual datasets after importation

,hh0,I,dI
180,-12.5487,0.0,0.0
181,-12.5462,0.0,0.0
182,-12.5437,0.0,0.0
183,-12.5412,0.0,0.0
184,-12.5388,0.0,0.0
185,-12.5363,0.0,0.0


## Basic data processing
Rescale data and remove "bad" data

### Check consistency of dataframes

#### <a name="hh0_consistency"></a>Check that all hh0 data are the same within each dataframe
i.e. that all hh0 of coarseData are the same and that all hh0 data of linecut_f_raw are the same

In [10]:
nData_raw = [coarseData,linecut_f_raw]
for data_idx in range(len(nData_raw)):# for each dataset
    for _, row in nData_raw[data_idx].iterrows():# loop over all rows
        if not np.array_equal(row.spectra.hh0,nData_raw[data_idx].spectra[0].hh0):
        # and compare the array of hh0 of that row with that of the first row
            print(row)# print the row if the two arrays are *not* equal
            # should output nothing, which means that all arrays of hh0 are the same *within a dataset*

#### Then check that hh0 data of coarseData differ from that of linecut_f_raw
We do not need to loop over all rows since we have shown in the previous cell that all rows are the same within a dataset

In [11]:
if not np.array_equal(coarseData.spectra[0].hh0,linecut_f_raw.spectra[0].hh0):
# compare the hh0 arrays of the first row of both datasets
    print("The arrarys of hh0 are not the same in both datasets")

The arrarys of hh0 are not the same in both datasets


### Which datasets to analyze and how? 

#### Update 2020-04-06
In fact, ignore the coarse dataset, as it will require a lot of efforts for a minimal result.
More interesting would be to analyze the linecut_f dataset at each (hh0) peak position, for h = 6, 8, 10.

##### Update 2020-04-03
treat both datasets independently in terms of the plotting and fitting

##### Outdated ideas
involving treating both datasets together, which will make things more complicated, and therefore increases the risks of making errors, in addition to increasing the time required for the analysis:
* interpolate spectra of coarseData so that its hh0 array is the same as that of linecut_f_raw
* rescale the data, if there is a physical way to do it, otherwise simply treat both datasets separately

### Truely "deep" copy of linecut_f
such that linecut_f_raw will *not* be modified if lcf_copy is modified

The best way to truely, i.e. recursively, deep copy a python object is to `pickle.dump` and `pickle.load` it. 
That is because `cPickle` is the fastest, as shown [here](https://stackoverflow.com/questions/1410615/copy-deepcopy-vs-pickle),  *and* **in Python 3**, `cPickle` is the default behavior of `pickle`, as explained [here](https://askubuntu.com/a/804618).
See also hacks.ipynb, and [here](https://stackoverflow.com/questions/52708341/make-a-truly-deep-copy-of-a-pandas-series). 

In [12]:
lcf_copy = pickle.loads(pickle.dumps(linecut_f_raw))# fastest python hack to create a truely deep copy
# lcf stands for linecut_f, which is the name of the file from which the data was imported
# idx = 5
lcf_copy.head()#.loc[idx:idx+10]

,filename,T (K),H (T),Proton charge,spectra
0,HH0_88631.txt,0.605973,0.000000,1.312139,hh0 I dI 0 -12.99880 0.0 ...
1,HH0_88632.txt,0.613334,0.049998,0.800119,hh0 I dI 0 -12.99880 0.0 ...
2,HH0_88633.txt,0.605727,0.099996,0.083534,hh0 I dI 0 -12.99880 0.0 ...
3,HH0_88634.txt,0.625493,0.099996,0.800739,hh0 I dI 0 -12.99880 0.0 ...
4,HH0_88635.txt,0.618220,0.150005,0.801294,hh0 I dI 0 -12.99880 0.0 ...


### Data rescaling
This used to be done manually, after noticing that the intensity of the data at 0T was higher than that of the rest of the data. A factor of 0.635 was then used to rescale this spectrum to the level of other data. 
However, after getting the up-to-date data, it appears that the scaling factor is merely the proton charge, which is a proxy for the counting time of neutrons. With this information, it turns out that the ratio of Proton charges of the spectrum at 0T and that at 0.05T is 1.3 to 0.8. The rescaling factor of the former is thus 0.8/1.3=0.615. Hence the empirical value of 0.635 was a pretty good guess!

In [13]:
##### Rescale data according to their Proton charge
for idx, row in lcf_copy.iterrows():
    row.spectra['Inorm'] = row.spectra.I/row['Proton charge']
    row.spectra['dInorm'] = row.spectra.dI/row['Proton charge']
#     print(idx, row['spectra']['I'])
idx = 1200# index that allows to look at data close to the (10 10 0) peak
lcf_copy.spectra[0].loc[idx:idx+5]# check one of the resulting dataframes

,hh0,I,dI,Inorm,dInorm
1200,-9.99875,1261600.0,94684.7,9.614833e+05,72160.557743
1201,-9.99625,1230390.0,98819.2,9.376977e+05,75311.519049
1202,-9.99375,1448420.0,109920.0,1.103862e+06,83771.596753
1203,-9.99125,2141070.0,124810.0,1.631740e+06,95119.477718
1204,-9.98875,2560080.0,141466.0,1.951073e+06,107813.252422
1205,-9.98625,4044660.0,169302.0,3.082493e+06,129027.464278


### Find bad data, if any
#### Identify datasets with zero intensity and create a new "clean" dataset without bad data

In [14]:
lcf_clean = lcf_copy.copy()
# this is a deepcopy according to Pandas, which is only deep at the lowest order, i.e. not recursively
for idx, row in lcf_copy.iterrows():# loop over all spectra
    if not np.any(row.spectra.Inorm>0):# if any spectrum has a constant zero intensity
        lcf_clean = lcf_copy.drop(idx)
        print(idx)# output the row index of lcf_copy containing empty data
# lcf_copy.spectra[0].Inorm

84


#### Batch plot the rest of the spectra to identify other potential bad data

In [15]:
%matplotlib qt
# plot figures in external window

##### Plot spectra by groups of nSpec
to see if any spectrum differs from the others.
This method may fail if there are more than nSpec bad consecutive spectra. Hence nSpec should not be too small (5 or more should be good)

In [16]:
nSpec = 7# number of curves in each plot
nfig = len(lcf_clean)//nSpec+1
fig = [None]*(nfig)
lgd = [None]*(nfig)
for fidx in range(1):#range(nfig):
    fig[fidx] = plt.figure()
    ax = fig[fidx].add_subplot(1,1,1)
    for pidx in range(nSpec*fidx,nSpec*(fidx+1)):
        try:
            plt.plot(lcf_clean.spectra[pidx].hh0,lcf_clean.spectra[pidx].Inorm,\
                     label=f"{pidx}, {lcf_clean['H (T)'][pidx]:.2f}")
        except KeyError:# if the index of data to be plotted does not exist
            continue# ignore and carry on to the next one
    plt.xlim(-10.25,-5.75)
    lgd[fidx] = ax.legend(title='Index, H (T)')
    lgd[fidx].set_draggable(True)
    plt.show()# ensures that all windows come to the foreground

##### Notes --- 2020-04-07
* After plotting all spectra together, it appears that only spectra #2 and #85 are "bad": the former is very noisy (not too surprising given that its Proton charge is 1/10 of the other data) and the latter has a lower intensity than the other spectra (for an unknown reason).
* Both those bad spectra were identified during the measurement, such that another (good) spectrum was measured at both values of magnetic field at which those bad spectra had been measured. Concretely, spectrum #2 was measured at 0.1T and #85 at 0.865T but are bad. Spectra #3 and #104 were also measured at 0.1T and 0.865T, respectively, and are good. Hence, the two bad spectra should just be discarded, there is no reason to try and process them in order to try and make them good: first because there is no way of making good data out of bad data (bad data is just bad data), so it would be a waste of time, and potentially a lot of time, but it would also be complicated, and because the result cannot be good, it can only influence the result of the subsequent fits in a bad way, thus inducing distrust on the results obtained over the whole dataset instead of just two spectra. Bottom line: discard spectra #2 and #85.

### Remove bad data 
as identified in batch plotting

In [17]:
delRowIdx = [2,85]# index of data to remove
lcf_clean.drop(delRowIdx,inplace=True)

### Final clean dataset
#### Sorted by value of magnetic field

In [18]:
nData = lcf_clean.sort_values(by=['H (T)'],ignore_index=True)
# lcf_clean.head()# nData stands for "neutrons Data"

#### Plot a couple of ENS spectra (ignore after done once)
to check consistency between spectra before fitting

### Plot entire dataset in 3D
as spectrum normalized intensity 'Inorm' vs position in reciprocal space 'hh0' and magnetic field 'H (T)', to check that the field dependence of the data is consistent.
#### Ignore after done once
Change cells to Raw type in order to avoid running inadvertantly

#### First plot individual spectra in a 3D space

#### Then plot 3D color map
##### Create meshgrid for 3D color map
The mesh can safely be created using hh0 data from the dataset measured at any value of magnetic field, since we've shown [that all hh0 data are the same within the linecut_f dataframe](#hh0_consistency)

##### Create the 2D array that contains the intensity data

##### Plot the hh0mesh and Hmesh data to see how irregular they are

##### Conclusion
The hh0 data looks pretty regular, however the magnetic field data is not
##### Update
That should not prevent from plotting in 3D

#### Use griddata to interpolate the intensity data over a regular array

#### Plot 3D surface
Note: Matplotlib does not do a very good job of plotting the 3D surface when the xlim is not adapted to the range of the data and when there is noise in the data: in our case, it looks like spectra are plotted individually rather than as a continuous surface. When zooming on each peak, the surface looks a little better. Perhaps look for a better 3D visualization tool.

#### Matplotlib example of 3D surface plotting

## Fit data

### Define fit result plotting function

In [20]:
def plot_fit_result(x, y, fitresult):
    """
    Plot result of fit using an lmfit Model object
    """
    fig = plt.figure()
    plt.plot(x, y, 'bo')
    plt.plot(x, fitresult.init_fit, 'k--', label='initial fit')
    plt.plot(x, fitresult.best_fit, 'r-', label='best fit')
    plt.legend(loc='best')
    plt.show()
    

#### Attempt to an iteration callback function
Does not work as of 2020-04-14

In [35]:
def iter_func(params, iter_num, resid, x):
    print(iter_num)
    return True

### Fit example adapted from the lmfit documentation 
(section "Modeling Data and Curve Fitting)

In [30]:
def gaussian(x, amp, cen, wid):
    return amp * np.exp(-(x-cen)**2 / wid)
gmodel = Model(gaussian)
print('parameter names: {}'.format(gmodel.param_names))
print('independent variables: {}'.format(gmodel.independent_vars))
gparams = gmodel.make_params(cen=5, amp=20, wid=1)
gparams

parameter names: ['amp', 'cen', 'wid']
independent variables: ['x']


name,value,initial value,min,max,vary
amp,20.0000000,None,-inf,inf,True
cen,5.00000000,None,-inf,inf,True
wid,1.00000000,None,-inf,inf,True


In [31]:
x = np.linspace(0,10,101)
noise = np.random.rand(101)-0.5
y = gaussian(x,amp=20,cen=5,wid=1)+noise
# fig = plt.figure()
# plt.plot(x, y)
# plt.show()
result = gmodel.fit(y, x=x, amp=5, cen=5, wid=1)
print(result.fit_report())
plot_fit_result(x, y, result)

[[Model]]
    Model(gaussian)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 17
    # data points      = 101
    # variables        = 3
    chi-square         = 8.43330605
    reduced chi-square = 0.08605414
    Akaike info crit   = -244.776096
    Bayesian info crit = -236.930735
[[Variables]]
    amp:  19.9229410 +/- 0.10155032 (0.51%) (init = 5)
    cen:  4.99454353 +/- 0.00415645 (0.08%) (init = 5)
    wid:  0.99742555 +/- 0.01174109 (1.18%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(amp, wid) = -0.577


### General parameters for our problem

In [21]:
peak_center = -8.0# center of unsplit peak to be studied in the following, in reciprocal space units
npeak_interval = .15# half of plot interval
Hc_0 = 0.51# value in Tesla units of the critical field at zero temperature
# in the absence of demagnetizing factor
# see data taken on needles of TmVO4-LS5200 in July 2017

### Single-peak fit with single pVIC function on single spectrum
measured above Hc, where the peaks are unsplit

#### Select data

In [25]:
dat_idx = 100
data_select = np.logical_and(nData.spectra[dat_idx].hh0 > peak_center - npeak_interval, 
                             nData.spectra[dat_idx].hh0 < peak_center + npeak_interval)
X = nData.spectra[dat_idx].hh0[data_select]# select data for first fit
fitdata = nData.spectra[dat_idx].Inorm[data_select]
X.head()

1940   -8.14875
1941   -8.14625
1942   -8.14375
1943   -8.14125
1944   -8.13875
Name: hh0, dtype: float64

#### Perform fit with Gaussian model as a first try

In [33]:
np.seterr(all='warn')# Set how floating-point errors are handled
# Setting warnings will help identify computation issues that may result in an unsuccessful fitting procedure
ngparams = gmodel.make_params(cen=peak_center, amp=4e6, wid=1e-3)
# Yeval = gmodel.eval(ngparams,x=X)# evaluate model using initial parameters
result = gmodel.fit(data=fitdata, params=ngparams, x=X)
print(result.fit_report())
plot_fit_result(X, fitdata, result)

[[Model]]
    Model(gaussian)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 33
    # data points      = 120
    # variables        = 3
    chi-square         = 2.6305e+12
    reduced chi-square = 2.2483e+10
    Akaike info crit   = 2863.28327
    Bayesian info crit = 2871.64575
[[Variables]]
    amp:  4232624.39 +/- 61044.4590 (1.44%) (init = 4000000)
    cen: -7.99724868 +/- 2.1258e-04 (0.00%) (init = -8)
    wid:  3.2588e-04 +/- 1.0854e-05 (3.33%) (init = 0.001)
[[Correlations]] (unreported correlations are < 0.100)
    C(amp, wid) = -0.577


#### Single pVIC fit model

##### Make model from fit function

In [108]:
from ENS_peak_fit_pVIC_py.pseudoVoigtIkedaCarpenter import pVIC, xpVIC_residual, xpVIC_prm_values
pvic_model = Model(pVIC)# create Model object from the lmfit module
print(f'parameter names: {pvic_model.param_names}')
print(f'independent variables: {pvic_model.independent_vars}')

parameter names: ['A', 'alpha', 'beta', 'R', 'gamma', 'sigma', 'k', 'xp']
independent variables: ['x']


#### Fit parameters
##### Create fit parameters and specify their properties
including initial values, constraints, etc.

In [27]:
pvic_params = pvic_model.make_params(A=2e5, alpha=140, beta=1e-3, R=1e-3, 
                                     gamma=1e-3, sigma=6.6e-3, k=.05, xp=peak_center)
# fitData = nData.spectra[0].Inorm
# pvic_params = pvic_model.guess(fitData)# returns NotImplementedError
for k in pvic_params.keys():
    pvic_params[k].set(min=0, vary=True)
#     pvic_params[k].init_value = pvic_params[k].value
#     print(k)
pvic_params['xp'].set(min=-np.inf)
pvic_params['k'].vary = False
pvic_params

name,value,initial value,min,max,vary
A,200000.000,None,0.00000000,inf,True
alpha,140.000000,None,0.00000000,inf,True
beta,1.0000e-03,None,0.00000000,inf,True
R,1.0000e-03,None,0.00000000,inf,True
gamma,1.0000e-03,None,0.00000000,inf,True
sigma,0.00660000,None,0.00000000,inf,True
k,0.05000000,None,0.00000000,inf,False
xp,-8.00000000,None,-inf,inf,True


#### Evaluate fit function

In [28]:
np.seterr(all='warn')# Set how floating-point errors are handled
Yeval = pvic_model.eval(pvic_params,x=X)# evaluate model using initial parameters
fig = plt.figure()
plt.plot(X,Yeval)
plt.plot(X,fitdata)
plt.show()

#### Next steps as of 2020-04-14
* Play with values of beta to get best fit result; in particular, let beta vary (with max value of 800?)
* Is there a way to output the adjusted R**2 of the fit?

#### Narrowing down the number of free parameters

##### 7 free parameters
all but k

In [47]:
pvic_params['beta'].set(value=1, max=.5e3, vary=True)
pvic_params
result = pvic_model.fit(data=fitdata, params=pvic_params, x=X)
print(result.fit_report())
plot_fit_result(X, fitdata, result)

[[Model]]
    Model(pVIC)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 295
    # data points      = 120
    # variables        = 7
    chi-square         = 2.8498e+11
    reduced chi-square = 2.5219e+09
    Akaike info crit   = 2604.58134
    Bayesian info crit = 2624.09378
[[Variables]]
    A:      141819.211 +/- 1069.17610 (0.75%) (init = 200000)
    alpha:  157.731938 +/- 41.0770286 (26.04%) (init = 140)
    beta:   161.301965 +/- 133.758272 (82.92%) (init = 1)
    R:      1.84262911 +/- 0.70370911 (38.19%) (init = 0.001)
    gamma:  0.00113801 +/- 2.3331e-04 (20.50%) (init = 0.001)
    sigma:  0.00639068 +/- 3.8955e-04 (6.10%) (init = 0.0066)
    k:      0.05 (fixed)
    x0:    -8.02486439 +/- 3.9278e-04 (0.00%) (init = -8)
[[Correlations]] (unreported correlations are < 0.100)
    C(alpha, beta)  = -0.996
    C(beta, R)      =  0.991
    C(alpha, R)     = -0.980
    C(A, gamma)     =  0.628
    C(A, alpha)     = -0.454
    C(A, beta)      =  0.452
 

The above yields too many correlations between parameters
The resulting plot also has an unphysical dip in the resulting fit function

##### 6 free parameters

In [66]:
pvic_params['beta'].set(value=1e-10, vary=False)
pvic_params['R'].set(value=0, vary=False)
result = pvic_model.fit(data=fitdata, params=pvic_params, x=X)
print(result.fit_report())
plot_fit_result(X, fitdata, result)
freeParams = [k for k in list(pvic_params.keys()) if pvic_params[k].vary==True]
plt.title(f"{len(freeParams)} free parameters: {freeParams}")

[[Model]]
    Model(pVIC)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 105
    # data points      = 120
    # variables        = 5
    chi-square         = 3.3204e+11
    reduced chi-square = 2.8873e+09
    Akaike info crit   = 2618.92398
    Bayesian info crit = 2632.86144
[[Variables]]
    A:      139800.970 +/- 832.210128 (0.60%) (init = 200000)
    alpha:  130.269301 +/- 1.82009419 (1.40%) (init = 140)
    beta:   1e-10 (fixed)
    R:      0 (fixed)
    gamma:  1.0779e-07 +/- 1.4438e-04 (133947.70%) (init = 0.001)
    sigma:  0.00596479 +/- 2.2440e-04 (3.76%) (init = 0.0066)
    k:      0.05 (fixed)
    x0:    -8.01731281 +/- 2.1916e-04 (0.00%) (init = -8)
[[Correlations]] (unreported correlations are < 0.100)
    C(alpha, x0)    =  0.942
    C(alpha, sigma) =  0.641
    C(sigma, x0)    =  0.631
    C(A, gamma)     =  0.472
    C(gamma, sigma) = -0.366
    C(A, x0)        = -0.227
    C(alpha, gamma) =  0.219
    C(A, alpha)     = -0.212
    C(A, sig

Text(0.5, 1.0, "5 free parameters: ['A', 'alpha', 'gamma', 'sigma', 'x0']")

### Batch fitting of single-peak spectra with single pVIC function
still above Hc

This will help fix the value of the parameters shared by all spectra, i.e. alpha, beta, R, gamma and sigma

#### Select datasets for batch fitting

In [123]:
data_select = np.logical_and(nData.spectra[dat_idx].hh0 > peak_center - npeak_interval, 
                             nData.spectra[dat_idx].hh0 < peak_center + npeak_interval)
# X = np.array([])
spec_rng = range(len(nData)-30, len(nData))
X = np.stack([nData.spectra[idx].hh0[data_select] for idx in spec_rng])
# x data for batch fitting
fitdata = np.stack([nData.spectra[idx].Inorm[data_select] for idx in spec_rng])

#### Parameters for batch fitting

In [124]:
xpvic_params = Parameters()
ref_idx = len(spec_rng)-2

# create new set of parameters for each dataset to fit
for spec_idx, _ in enumerate(X):
    for k, v in pvic_params.items():
        if k in ['A', 'xp']:# fit parameters that are different for each dataset
            par_key = f'{k}{spec_rng[spec_idx]}'
            xpvic_params.add( par_key, value=pvic_params[k].value, 
                              min=pvic_params[k].min, vary=pvic_params[k].vary )
        else:# all other parameters are shared by all datasets
            xpvic_params[k] = pvic_params[k]

# then assign expression for all dependent parameters
# for spec_idx, _ in enumerate(X):
#     for k, _ in pvic_params.items():
#         if spec_idx!=ref_idx and k not in ['A', 'xp']:
#             par_key = f'{k}{spec_rng[spec_idx]}'
#             xpvic_params[par_key].expr = xpvic_params[f'{k}{spec_rng[ref_idx]}'].name
            # Set parameters alpha, beta, R, gamma and sigma to be have the same value across all datasets
#             print(k, xpvic_params[0][f'{k}{spec_rng[0]}'].name)

#### Perform fit on multiple functions
and extract the resulting fit parameters

In [125]:
# run the global fit to all the data sets
result = minimize(xpVIC_residual, xpvic_params, args=(X, fitdata, spec_rng))
report_fit(result)

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 1992
    # data points      = 3600
    # variables        = 65
    chi-square         = 1.1437e+13
    reduced chi-square = 3.2353e+09
    Akaike info crit   = 78895.0432
    Bayesian info crit = 79297.3080
[[Variables]]
    A72:    163933.234 +/- 723.464381 (0.44%) (init = 200000)
    alpha:  136.223295 +/- 1.71547974 (1.26%) (init = 140)
    beta:   294.435168 +/- 68.0504689 (23.11%) (init = 0.001)
    R:      0.96913976 +/- 78.4522422 (8095.04%) (init = 0.001)
    gamma:  6.4333e-04 +/- 1.0156e-04 (15.79%) (init = 0.001)
    sigma:  0.00572939 +/- 0.00475011 (82.91%) (init = 0.0066)
    k:      0.05 (fixed)
    xp72:  -8.01927176 +/- 0.26564398 (3.31%) (init = -8)
    A73:    159822.399 +/- 709.868084 (0.44%) (init = 200000)
    xp73:  -8.01911240 +/- 0.26564802 (3.31%) (init = -8)
    A74:    161290.936 +/- 714.911524 (0.44%) (init = 200000)
    xp74:  -8.01931880 +/- 0.26564653 (3.31%) (init = -8)
    A75

In [122]:
result.params

name,value,standard error,relative error,initial value,min,max,vary
A97,141965.235,710.794933,(0.50%),200000.0,0.00000000,inf,True
alpha,129.962820,1.58094390,(1.22%),140,0.00000000,inf,True
beta,1631.12245,196550.743,(12050.03%),0.001,0.00000000,inf,True
R,0.46968212,333.505179,(71006.57%),0.001,0.00000000,inf,True
gamma,3.3713e-07,1.0899e-04,(32329.72%),0.001,0.00000000,inf,True
sigma,0.00596256,0.00596609,(100.06%),0.0066,0.00000000,inf,True
k,0.05000000,0.00000000,(0.00%),0.05,0.00000000,inf,False
xp97,-8.01724670,0.17056245,(2.13%),-8.0,-inf,inf,True
A98,142237.024,736.742867,(0.52%),200000.0,0.00000000,inf,True
xp98,-8.01726075,0.17053254,(2.13%),-8.0,-inf,inf,True


In [118]:
result.params

name,value,standard error,relative error,initial value,min,max,vary
A100,145187.046,49691.7213,(34.23%),200000.0,0.00000000,inf,True
alpha,129.525829,1.95388472,(1.51%),140,0.00000000,inf,True
beta,2.20185045,24.4762840,(1111.62%),0.001,0.00000000,inf,True
R,0.03781341,0.32356877,(855.70%),0.001,0.00000000,inf,True
gamma,2.3248e-08,1.4107e-05,(60677.51%),0.001,0.00000000,inf,True
sigma,0.00591827,1.9159e-04,(3.24%),0.0066,0.00000000,inf,True
k,0.05000000,0.00000000,(0.00%),0.05,0.00000000,inf,False
xp100,-8.01742397,2.1295e-04,(0.00%),-8.0,-inf,inf,True
A101,141510.461,48436.6725,(34.23%),200000.0,0.00000000,inf,True
xp101,-8.01732648,2.1319e-04,(0.00%),-8.0,-inf,inf,True


In [127]:
bestparams = np.zeros((len(spec_rng),8))# shape = # of datasets x # of parameters in fit function (pVIC)
for spec_idx in range(len(spec_rng)):
    for par_idx, k in enumerate(pvic_params.keys()):
        par_key = f'{k}{spec_rng[spec_idx]}' 
        # parameter name is a concatenation of the generic parameter name,
        # as defined in the pVIC function, and the spectrum index
        try:
            bestparams[spec_idx][par_idx] = result.params[par_key].value
        except KeyError:
            bestparams[spec_idx][par_idx] = result.params[k].value
        
# list(result.params.valuesdict().values())
# bestparams

As already noted with the analysis in Matlab, the values of beta and R are completely irrelevant.
Extend the fitting procedure to Hc+ just to make sure that it doesn't change anything, but don't waste too much time on that. 
Then, fix value of beta and R to zero and move on to determining the common values of all other parameters that are common to all dataset, i.e. alpha, gamma and sigma.

In [128]:
fig = plt.figure()
for idx, spec_idx in enumerate(spec_rng):
    bestfit = pVIC(X[idx], *bestparams[idx][:8])
    plt.plot(X[idx], fitdata[idx], 'o', label=f"expt {nData['H (T)'][spec_idx]:.2f}T")
    plt.plot(X[idx], bestfit, '-', label=f"fit {nData['H (T)'][spec_idx]:.2f}T")
    plt.legend(loc='best')
    plt.show()
freeParams = [k for k in list(xpvic_params.keys()) if xpvic_params[k].vary==True]
plt.title(f"TmVO$_4$ neutrons {len(freeParams)} free parameters")

Text(0.5, 1.0, 'TmVO$_4$ neutrons 65 free parameters')